#Import libraries

In [ ]:
# import numpy, pandas and other necessary libraries
import re
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [ ]:
import zipfile
from google.colab import drive

drive.mount('/content/drive/')



Mounted at /content/drive/


#Preprocessing

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/US-accident/US_Accidents_Dec21_updated.csv')
data.head()

,ID,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,3,2016-02-08 00:37:08,2016-02-08 06:37:08,40.108910,-83.092860,40.112060,-83.031870,3.230,Between Sawmill Rd/Exit 20 and OH-315/Olentang...,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,2,2016-02-08 05:56:20,2016-02-08 11:56:20,39.865420,-84.062800,39.865010,-84.048730,0.747,At OH-4/OH-235/Exit 41 - Accident.,...,False,False,False,False,False,False,Night,Night,Night,Night
2,A-3,2,2016-02-08 06:15:39,2016-02-08 12:15:39,39.102660,-84.524680,39.102090,-84.523960,0.055,At I-71/US-50/Exit 1 - Accident.,...,False,False,False,False,False,False,Night,Night,Night,Day
3,A-4,2,2016-02-08 06:51:45,2016-02-08 12:51:45,41.062130,-81.537840,41.062170,-81.535470,0.123,At Dart Ave/Exit 21 - Accident.,...,False,False,False,False,False,False,Night,Night,Day,Day
4,A-5,3,2016-02-08 07:53:43,2016-02-08 13:53:43,39.172393,-84.492792,39.170476,-84.501798,0.500,At Mitchell Ave/Exit 6 - Accident.,...,False,False,False,False,False,False,Day,Day,Day,Day


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2845342 entries, 0 to 2845341
Data columns (total 47 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   ID                     object 
 1   Severity               int64  
 2   Start_Time             object 
 3   End_Time               object 
 4   Start_Lat              float64
 5   Start_Lng              float64
 6   End_Lat                float64
 7   End_Lng                float64
 8   Distance(mi)           float64
 9   Description            object 
 10  Number                 float64
 11  Street                 object 
 12  Side                   object 
 13  City                   object 
 14  County                 object 
 15  State                  object 
 16  Zipcode                object 
 17  Country                object 
 18  Timezone               object 
 19  Airport_Code           object 
 20  Weather_Timestamp      object 
 21  Temperature(F)         float64
 22  Wind_Chill(F)     

In [ ]:
# Displaying the number of missing values per attribute

print('Percentage of Missing values:\n\n',(100*data.isnull().sum()/data.shape[0]).round(2))

Percentage of Missing values:

 ID                        0.00
Severity                  0.00
Start_Time                0.00
End_Time                  0.00
Start_Lat                 0.00
Start_Lng                 0.00
End_Lat                   0.00
End_Lng                   0.00
Distance(mi)              0.00
Description               0.00
Number                   61.29
Street                    0.00
Side                      0.00
City                      0.00
County                    0.00
State                     0.00
Zipcode                   0.05
Country                   0.00
Timezone                  0.13
Airport_Code              0.34
Weather_Timestamp         1.78
Temperature(F)            2.43
Wind_Chill(F)            16.51
Humidity(%)               2.57
Pressure(in)              2.08
Visibility(mi)            2.48
Wind_Direction            2.59
Wind_Speed(mph)           5.55
Precipitation(in)        19.31
Weather_Condition         2.48
Amenity                   0.00
Bump   

In [ ]:
#Displaying the number of unique values per attribute

print('Unique values per column :',data.nunique())



Unique values per column : ID                       2845342
Severity                       4
Start_Time               1959333
End_Time                 2351505
Start_Lat                1093618
Start_Lng                1120365
End_Lat                  1080811
End_Lng                  1105404
Distance(mi)               14165
Description              1174563
Number                     46402
Street                    159651
Side                           3
City                       11681
County                      1707
State                         49
Zipcode                   363085
Country                        1
Timezone                       4
Airport_Code                2004
Weather_Timestamp         474214
Temperature(F)               788
Wind_Chill(F)                897
Humidity(%)                  100
Pressure(in)                1068
Visibility(mi)                76
Wind_Direction                24
Wind_Speed(mph)              136
Precipitation(in)            230
Weather_Conditio

In [ ]:
# Removing the attributes from the dataset

data.drop(columns=['Number'], inplace=True)

In [ ]:
# Setting 'Wind_Speed(mph)' as 0 for rows where 'Wind_Direction' is 'Calm', meaning almost no wind.

data.loc[data['Wind_Direction'] == 'Calm', 'Wind_Speed(mph)'] = 0

In [ ]:
#replace nan with median for precipitation
data['Precipitation_NA'] = 0
data.loc[data['Precipitation(in)'].isnull(),'Precipitation_NA'] = 1
data['Precipitation(in)'] = data['Precipitation(in)'].fillna(data['Precipitation(in)'].median())
data.loc[:5,['Precipitation(in)','Precipitation_NA']]

,Precipitation(in),Precipitation_NA
0,0.00,0
1,0.02,0
2,0.02,0
3,0.00,0
4,0.01,0
5,0.00,0


In [ ]:
#replace nan with median for wind_Chill
data['wind_chill_na'] = 0
data.loc[data['Wind_Chill(F)'].isnull(),'wind_chill_na'] = 1
data['Wind_Chill(F)'] = data['Wind_Chill(F)'].fillna(data['Wind_Chill(F)'].median())
data.loc[:5,['Wind_Chill(F)','wind_chill_na']]

,Wind_Chill(F),wind_chill_na
0,36.1,0
1,63.0,1
2,63.0,1
3,63.0,1
4,29.8,0
5,29.2,0


In [ ]:
data.dropna(inplace=True)

In [ ]:
# Converting datatype for attributes related to datetime.

data["Start_Time"]= pd.to_datetime(data["Start_Time"])
data["End_Time"]= pd.to_datetime(data["End_Time"])
data["Weather_Timestamp"]= pd.to_datetime(data["Weather_Timestamp"])

In [ ]:
# average difference between weather time and start time
print("Mean difference between 'Start_Time' and 'Weather_Timestamp': ",
(data.Weather_Timestamp - data.Start_Time).mean())



Mean difference between 'Start_Time' and 'Weather_Timestamp':  0 days 00:01:10.419258574


In [ ]:
data = data.drop(["Weather_Timestamp"], axis=1)

In [ ]:
# Checking the percentage of Duplicate records in the dataset

print(f'Percentage of duplicate records: {100-(100*data.drop_duplicates().shape[0]/data.shape[0])}')

Percentage of duplicate records: 0.0


In [ ]:
# Adding consistency to the various wind speed direction values

data['Wind_Direction'].replace({'North': 'N'}, inplace=True)
data['Wind_Direction'].replace({'East': 'E'}, inplace=True)
data['Wind_Direction'].replace({'West': 'W'}, inplace=True)
data['Wind_Direction'].replace({'South': 'S'}, inplace=True)
data['Wind_Direction'].replace({'VAR': 'Variable'}, inplace=True)
data['Wind_Direction'].replace({'CALM': 'Calm'}, inplace=True)

data['Wind_Direction'].unique()



array(['SW', 'Calm', 'WSW', 'WNW', 'W', 'NNW', 'S', 'NW', 'N', 'SSE',
       'SSW', 'ESE', 'SE', 'E', 'Variable', 'NNE', 'NE', 'ENE'],
      dtype=object)

In [ ]:
# 'End_Time' should always be greater than 'Start_Time'

data.drop(data[data['End_Time']<data['Start_Time']].index, inplace=True)



In [ ]:
#check catergorical features
cat_names = ['Side', 'Country', 'Timezone', 'Amenity', 'Bump', 'Crossing',
             'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station',
             'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop', 'Sunrise_Sunset',
             'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight']
print("Unique count of categorical features:")
for i in cat_names:
  print(i,data[i].unique().size)

In [ ]:
data = data.drop(['Country','Turning_Loop', 'ID'], axis=1)

In [ ]:
# Deriving the attribute 'Time_Duration(min)'

data.insert(4,'Time_Duration(min)',(data['End_Time']-data['Start_Time'])//np.timedelta64(1,'m'))

In [ ]:
data['Time_Duration(min)'].describe()

count    2.731064e+06
mean     3.548349e+02
std      9.260334e+03
min      2.000000e+00
25%      7.300000e+01
50%      1.200000e+02
75%      2.230000e+02
max      1.682579e+06
Name: Time_Duration(min), dtype: float64

In [ ]:
# Dropping the attribute 'End_Time' since it is redundant now

data.drop(columns=['End_Time'], inplace=True)

In [ ]:
# Breaking 'Start_Time' into 'Year', 'Month', 'Day', 'Hour' and 'Weekend'

data['Year']=data['Start_Time'].dt.year
data['Month']=data['Start_Time'].dt.month
data['Day']=data['Start_Time'].dt.day
data['Hour']=data['Start_Time'].dt.hour
data['Minute']=data['Start_Time'].dt.minute
data['Weekday']=data['Start_Time'].dt.weekday

def weekday_text(w):
    d = {0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday', 4:'Friday', 5:'Saturday', 6:'Sunday'}
    return d[w]
data['Weekday']=data['Weekday'].apply(lambda x:weekday_text(x))

In [ ]:
# Dropping the attribute 'Start_Time' since it is redundant now

data.drop(columns=['Start_Time'], inplace=True)

In [ ]:
# Removing outliers | Time_Duration
# Removing records stating duration more than 12 days (since higher than 12 days is not recorded yet)

data.drop(data[data['Time_Duration(min)'] > (12*1440)].index, inplace=True)

In [ ]:
# Removing outliers | Wind_Speed(mph)
# Removing records wind speed more than 260 mph (since higher than ~253mph is not recorded yet)

data.drop(data[data['Wind_Speed(mph)'] > 260].index, inplace=True)

In [ ]:
# Removing outliers | Distance(mi)
# Removing records distance(mi) more than 109 miles (since higher than ~109 miles is not recorded yet)

data.drop(data[data['Distance(mi)'] > 109].index, inplace=True)




In [ ]:
# Removing outliers | Temperature(F)
# Removing records temperature(f) more than 131.4 mph (since higher than ~134.1 F is not recorded yet)

data.drop(data[data['Temperature(F)'] > 134.1].index, inplace=True)



In [ ]:
# Removing outliers | Pressure(in)
# Removing records pressure(in) less than 25.69 (since lesser than ~25.69 is not recorded yet)
# Removing records pressure(in) more than 32.03 (since higher than ~32.03 is not recorded yet)

data.drop(data[data['Pressure(in)'] < 25.69].index, inplace=True)
data.drop(data[data['Pressure(in)'] > 32.03].index, inplace=True)



In [ ]:
# Removing outliers | Wind_Speed(mph)
# Removing records visibility(mi) more than 150 miles (since higher than ~150 miles is not recorded yet)

data.drop(data[data['Visibility(mi)'] > 150].index, inplace=True)

In [ ]:
# Checking if none of the value is NaN and all of the values are finite and saving to file

if data.notnull().values.all() and not data.isnull().values.any():
    data.to_csv("/content/drive/MyDrive/US-accident-data", index=False)
    print('Data Saved')
else:
    print('Data Not Saved')

Data Saved


In [ ]:
# List of selected attributes based upon fast availability of attributes, considering:
# 2. the non repetition of information (e.g. Complete Address + Zipcode)

cols = [
        'Severity', 'Start_Lat', 'Start_Lng', 'Temperature(F)',
        'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Direction', 'Wind_Speed(mph)', 'Weather_Condition',
        'Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway','Roundabout', 'Station',
        'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Sunrise_Sunset', 'Civil_Twilight',
        'Year', 'Month', 'Day', 'Hour', 'Minute','Weekday',
        'Street', 'Side', 'City', 'County','Description'
]

In [ ]:
# Saving the top state specific dataset to disk and freeing up RAM before building model

# State CA
data[data['State']=='CA'][cols].to_csv("/content/drive/MyDrive/US-accident-data_CA", index=False)
